In [18]:
!curl https://norvig.com/ngrams/count_1w.txt -o count_1w.txt 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4840k  100 4840k    0     0  11.5M      0 --:--:-- --:--:-- --:--:-- 11.5M


In [19]:
!curl https://raw.githubusercontent.com/first20hours/google-10000-english/master/google-10000-english-no-swears.txt -o top-10000-no-swears.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75153  100 75153    0     0   396k      0 --:--:-- --:--:-- --:--:--  394k


In [3]:
with open('top-10000-no-swears.txt', 'r') as f:
  safe_words = set(list(f.read().splitlines()))

In [4]:
len(safe_words)

9894

In [5]:
word_to_count = {}

with open('count_1w.txt', 'r') as f:
  for x in f.readlines():
    word, count = x.split()
    if word not in safe_words:
      continue
    if len(word) < 3 or len(word) > 6:
      continue

    invalid = False
    for i in range(len(word)-1):
      if word[i] == word[i+1]:
        invalid = True
        break
    if invalid:
      continue
    word_to_count[word] = int(count)

In [21]:
LINE_SIZE = 7
DICTIONARY_SIZE = 1000
WORD_SIZE = 6
FIG_SIZE = 4
SIDE_SIZE = 3

In [7]:
!curl https://raw.githubusercontent.com/dwyl/english-words/master/words_dictionary.json -o full_dictionary.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6665k  100 6665k    0     0  15.1M      0 --:--:-- --:--:-- --:--:-- 15.1M


In [8]:
import json
  
with open('full_dictionary.json') as json_file:
    full_dictionary = json.load(json_file)

def is_generateable(word, figure):
  if len(word) > 6 or len(word) < 3:
    return False
  for i in range(len(word)-1):
    cando = False
    for side_j in figure:
      for side_k in figure:
        if side_j == side_k:
          continue
        if word[i] in side_j and word[i+1] in side_k:
          cando = True
          break
    if not cando:
      return False
  return True

def get_full_dict(figure):
  seed_dict = []
  for word in full_dictionary.keys():
    if is_generateable(word, figure):
      seed_dict.append(word)

  return seed_dict

In [ ]:
import random
import numpy as np
import string
from copy import deepcopy
import time

rolls = 0
st = time.time()
while True:
  rolls += 1
  if rolls % 100 == 0:
    print("Rolls: {}, Elapsed time: {}".format(rolls, time.time()-st))
  ch = random.choice(string.ascii_lowercase)
  res = [ []*3 for i in range(4)]
  prev_side = random.choice(list(range(4)))
  res[prev_side].append(ch)
  charset = set([ch])
  past_words = set()

  def can_use(chosen_word):
    global res, prev_side, charset
    modded_res = deepcopy(res)
    modded_prev_side = prev_side
    modded_charset = deepcopy(charset)
    is_first_letter = True
    for char in chosen_word:
      if char not in modded_charset:
        found_side = None
        side_ordering = np.random.permutation(list(range(4)))
        for side in side_ordering:
          if side == modded_prev_side:
            continue
          if len(modded_res[side]) == 3:
            continue
          found_side = side
          break
        if found_side is None:
          return False
        modded_res[found_side].append(char)
        modded_prev_side = found_side
        modded_charset.add(char)
      else:
        for i in range(4):
          if char in modded_res[i]:
            # print(char, i)
            if i == modded_prev_side and not is_first_letter:
              return False
            modded_prev_side = i
      is_first_letter = False
    res = modded_res
    prev_side = modded_prev_side
    charset = modded_charset
    past_words.add(chosen_word)
    return True


  chosen_word_soln = []
  while len(charset) < 12 and len(past_words) < 5:
    filtered_words = list(filter(lambda x: x.startswith(ch) and x not in past_words, list(word_to_count.keys())))
    filtered_weights = [word_to_count[word] for word in filtered_words]
    sum_w = sum(filtered_weights)
    filtered_weights = [x/sum_w for x in filtered_weights]
    chosen_word = None
    iter = 0
    while (chosen_word is None or not can_use(chosen_word)) and iter < 100:
      chosen_word = np.random.choice(filtered_words, p=filtered_weights)
      iter += 1
    
    # print(chosen_word, word_to_count[chosen_word])
    chosen_word_soln.append(chosen_word)
    ch = chosen_word[-1]

  def is_figure_nice(figure):
    if len(figure) != FIG_SIZE:
      return False
    for side in figure:
      if len(side) != SIDE_SIZE:
        return False
    
    return True

  figure = list([list(np.random.permutation(x)) for x in res])
  if not is_figure_nice(figure):
    continue
  dict_size = len(get_full_dict(figure))
  if dict_size > DICTIONARY_SIZE:
    continue
  print("Natural soln: {}".format(chosen_word_soln))
  print("Figure: {}".format(figure))
  print("Dict size: {}".format(dict_size))

In [10]:
# bad game seeds
bad_seeds = [
    {
        'figure': [['e', 'o', 'a'], ['n', 'l', 't'], ['r', 's', 'g'], ['v', 'i', 'd']],
        'solution': ['starting', 'gold', 'design', 'national', 'live'],
    },
    {
        'figure': [['c', 'e', 'y'], ['p', 'l', 'o'], ['i', 'u', 't'], ['v', 'm', 'r']],
        'solution': ['improve', 'electric', 'city', 'your'],
    },
    {
        'figure': [['i', 'n', 'e'], ['m', 'r', 's'], ['t', 'u', 'f'], ['o', 'd', 'l']],
        'solution': ['for', 'results', 'stories', 'some', 'end'],
    },
    {
        'figure': [['a', 's', 'e'], ['c', 'i', 'p'], ['t', 'l', 'n'], ['b', 'd', 'm']],
        'solution': ['dance', 'email', 'labs', 'split'],
    },    
    {
        'figure': [['l', 'c', 'f'], ['i', 'o', 'h'], ['b', 'p', 'e'], ['u', 'r', 'a']],
        'solution': ['blue', 'each', 'help', 'profile'],
    }
]

In [44]:
# good game seeds
good_seeds = [
    {
        'figure': [['l', 'u', 'k'], ['r', 'm', 'i'], ['s', 'c', 'n'], ['o', 'g', 'f']],
        'line': ['from', 'music', 'click', 'king'],
    },
    {
        'figure': [['i', 'y', 'n'], ['k', 'v', 'u'], ['t', 'e', 'r'], ['o', 'c', 'h']],
        'line': ['the', 'entry', 'your', 'river', 'rock'],
    },
]

for seed in good_seeds:
  seed['dict'] = get_full_dict(seed['figure'])

In [48]:
def get_poly_hash(wordlist):
  res = 0
  for val in wordlist:
    res = res*32 + val
  return res

def string_to_hash(word, req_len):
  res = []
  for ch in word:

    res.append(ord(ch) - ord('a'))

  while len(res) < req_len:
    res.append(27)
  return get_poly_hash(res)

def get_computer_readable_form(seed):
  # Verify conditions
  assert len(seed['figure']) == FIG_SIZE
  for side in seed['figure']:
    assert len(side) == SIDE_SIZE
  
  assert len(seed['dict']) <= DICTIONARY_SIZE
  for word in seed['dict']:
    assert len(word) <= WORD_SIZE

  assert len(seed['line']) <= LINE_SIZE
  
  def get_padding(req_len):
    return get_poly_hash([28]*req_len)  
  comp_dictionary = [string_to_hash(word, WORD_SIZE) for word in seed['dict']]
  while len(comp_dictionary) < DICTIONARY_SIZE:
    comp_dictionary.append(get_padding(WORD_SIZE))
  
  comp_line = [string_to_hash(word, WORD_SIZE) for word in seed['line']]
  while len(comp_line) < LINE_SIZE:
    comp_line.append(get_padding(WORD_SIZE))          
  return {
      'dictionary': comp_dictionary,
      'line': comp_line,
  }

In [49]:
import json

comp_form = get_computer_readable_form(good_seeds[0])

with open("seed.json", "w") as outfile:
    json.dump(comp_form, outfile)

In [50]:
string_to_hash('from', 6)

186069883